In [1]:
import pandas as pd

# 경로 설정
file_path = "gs://codeit-project/hackle/2023-05-24.parquet"
key_path = "/home/leesh/codeit_project/codeit-project-docker/config/key.json"

# GCS에서 parquet 불러오기
df_230524 = pd.read_parquet(file_path, storage_options={"token": key_path})

# 확인
print(df_230524.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2586328 entries, 0 to 2586327
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Asia/Seoul         object 
 1   event_key          object 
 2   session_id         object 
 3   user_id            object 
 4   value              float64
 5   user_properties    object 
 6   hackle_properties  object 
 7   event_properties   object 
 8   id                 object 
 9   device_id          object 
dtypes: float64(1), object(9)
memory usage: 197.3+ MB
None


In [2]:
df_230524.head()

,Asia/Seoul,event_key,session_id,user_id,value,user_properties,hackle_properties,event_properties,id,device_id
0,2023-05-25T00:00:00.219Z,$session_start,TeeVVRGrpuXE5q6f7pAu9ULVNvG3,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400219.62ee4573'},75b35f77-565e-47d2-b12e-91848a30b6e4,b6afceaa-cc9c-4a66-b92f-9e202f9eb328
1,2023-05-25T00:00:00.224Z,$session_start,9tcpOeHvyldCUXMgRyFVQI86rf13,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400224.df939c47'},20a73b83-a331-4d0f-8d96-09ccb796ba4e,54a8af4d-47bf-4951-87c2-1817b26d1c31
2,2023-05-25T00:00:00.275Z,$session_start,aiDrtN8uHiQXRXmLMJhf27hKA4B3,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400275.457b984b'},eb282c48-17e1-46b7-badd-8a2145901ffe,b08ee1df-0bc6-4b38-9474-8cbc55fd6f17
3,2023-05-25T00:00:00.369Z,$session_start,Oqmt3cCrCQZAa0TT3B4s2kTOG1k1,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400369.db2cd3ea'},f466ccc8-f453-4008-b319-47a28ae844f9,ace73d0e-04d6-4d03-9460-a0a1143253f7
4,2023-05-25T00:00:00.493Z,$session_start,GA1Nz2ehK2TmsRcqBzXQn6hkDHH2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400493.8e886f45'},a7c5252f-0969-423d-8098-3372c9614a47,2adda5ce-1d37-43cd-8cbd-c75c06f84f74


In [3]:
# 주요 정보 추출
df_simple = df_230524.copy()

# 딕셔너리 → Series로 분해 (중요한 컬럼만)
df_simple['language'] = df_simple['hackle_properties'].apply(lambda x: x.get('language'))
df_simple['devicemodel'] = df_simple['hackle_properties'].apply(lambda x: x.get('devicemodel'))
df_simple['sessionid'] = df_simple['event_properties'].apply(lambda x: x.get('sessionid'))


# 1. UTC 기준으로 파싱
df_simple['event_datetime'] = pd.to_datetime(df_simple['Asia/Seoul'], utc=True)

# 2. 한국시간(Asia/Seoul)으로 타임존 변환
df_simple['event_datetime'] = df_simple['event_datetime'].dt.tz_convert('Asia/Seoul')

# 최종 확인용 컬럼만 선택
df_simple = df_simple[['event_datetime','session_id','device_id', 'event_key', 'devicemodel', 'language']]

In [4]:
df_simple['event_datetime'] = df_simple['event_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_simple.head()

,event_datetime,session_id,device_id,event_key,devicemodel,language
0,2023-05-25 09:00:00,TeeVVRGrpuXE5q6f7pAu9ULVNvG3,b6afceaa-cc9c-4a66-b92f-9e202f9eb328,$session_start,"iPhone14,7",ko-KR
1,2023-05-25 09:00:00,9tcpOeHvyldCUXMgRyFVQI86rf13,54a8af4d-47bf-4951-87c2-1817b26d1c31,$session_start,"iPhone12,1",ko-KR
2,2023-05-25 09:00:00,aiDrtN8uHiQXRXmLMJhf27hKA4B3,b08ee1df-0bc6-4b38-9474-8cbc55fd6f17,$session_start,"iPhone12,1",ko-KR
3,2023-05-25 09:00:00,Oqmt3cCrCQZAa0TT3B4s2kTOG1k1,ace73d0e-04d6-4d03-9460-a0a1143253f7,$session_start,"iPhone11,8",ko-KR
4,2023-05-25 09:00:00,GA1Nz2ehK2TmsRcqBzXQn6hkDHH2,2adda5ce-1d37-43cd-8cbd-c75c06f84f74,$session_start,"iPhone14,7",ko-KR


In [5]:
df_simple.to_parquet("/home/leesh/codeit_project/codeit-project-docker/parquet/230524.parquet", engine="pyarrow", compression="snappy")